# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv('../WeatherPy/city_weather.csv')

In [3]:
city_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bengkulu,-3.80,102.27,75.33,89,55,3.40,ID,1602878570
1,1,constitucion,-35.33,-72.42,59.31,75,5,6.40,CL,1602878571
2,2,jalu,29.03,21.55,83.10,20,9,10.22,LY,1602878572
3,3,dikson,73.51,80.55,25.30,95,100,11.45,RU,1602878540
4,4,victoria,22.29,114.16,77.00,73,40,10.78,HK,1602878351
...,...,...,...,...,...,...,...,...,...,...
452,452,shingu,33.73,135.98,57.94,91,100,7.83,JP,1602879110
453,453,buta,2.79,24.73,68.63,96,25,0.65,CD,1602879111
454,454,recco,44.36,9.14,55.99,86,19,4.90,IT,1602879112
455,455,sinnamary,5.38,-52.95,83.98,66,0,13.73,GF,1602879113


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Plot heatmap
fig = gmaps.figure()

# Create heat layer
locations = city_weather[["Lat","Lng"]]
weight = city_weather["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=weight)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Filter cities based on ideal weather conditions
filter_criteria = ((city_weather["Max Temp"]>75) &
            (city_weather["Max Temp"]<=89) &
            (city_weather["Humidity"]<85) &
            (city_weather["Wind Speed"]<13) &
            (city_weather["Cloudiness"]<=40))

vac_cities = city_weather[filter_criteria]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = vac_cities.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

In [10]:
# get lat,lng from DataFrame
for index, row in hotel_df.iterrows():
    try:      
        # API base URL and parameters
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        # add params dictionary to update iterations
        params = {
            "type":"lodging",
            "keyword":"hotel",
            "radius":5000,
            "key":g_key,
            }
    
        city = row["City"]
        lat = row["Lat"]
        lon = row["Lng"]
    
    
        params['location'] = f"{lat},{lon}"
        
        response = requests.get(base_url,  params=params).json()
        results = response['results']
        print(f"A hotel in {city} was found.")  
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
            
    except:
        print("Hotel not found ... skipping.")

A hotel in jalu was found.
A hotel in victoria was found.
A hotel in boisar was found.
A hotel in nicoya was found.
A hotel in arcata was found.
A hotel in bobo dioulasso was found.
A hotel in waverley was found.
A hotel in bilma was found.
Hotel not found ... skipping.
A hotel in pisco was found.
A hotel in saint george was found.
A hotel in aswan was found.
A hotel in hilo was found.
A hotel in samana was found.
A hotel in luganville was found.
A hotel in avera was found.
Hotel not found ... skipping.
A hotel in salalah was found.
A hotel in bushehr was found.
A hotel in bardiyah was found.
A hotel in yulara was found.
A hotel in henderson was found.
Hotel not found ... skipping.
A hotel in pali was found.
A hotel in tuxpan was found.
A hotel in tres passos was found.
A hotel in port keats was found.
A hotel in north bend was found.
A hotel in lamu was found.
A hotel in carutapera was found.
A hotel in santa maria was found.
A hotel in trelew was found.
A hotel in iquitos was found.


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,brandon,US,27.94,-82.29,Sheraton Tampa Brandon Hotel
54,san quintin,MX,30.48,-115.95,Old Mill Hotel
62,pacific grove,US,36.62,-121.92,Monterey Plaza Hotel & Spa
69,faanui,PF,-16.48,-151.75,InterContinental Bora Bora Resort Thalasso Spa
74,salalah,OM,17.02,54.09,Al Jabal
81,quixada,BR,-4.97,-39.02,Hotel Vale das Pedras
84,bilma,NE,18.69,12.92,
93,mitsamiouli,KM,-11.38,43.28,Al Camar Lodge
96,teseney,ER,15.11,36.66,Ferait Hotel
118,tindwari,IN,25.62,80.53,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
    
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))